In [ ]:
from functions import MAP_THIGH, MAP_THIGH_TRUNK, process_motus

_, thigh_trunk_adults = process_motus(
    'thigh+trunk_adults',
    vendor='Other',
    map=MAP_THIGH_TRUNK,
    trunk_sensor=True,
    rotate=False,
    orientation=False,
)

_, thigh_trunk_older_adults = process_motus(
    'thigh+trunk_older_adults',
    vendor='Other',
    map=MAP_THIGH_TRUNK,
    trunk_sensor=True,
    rotate=False,
    orientation=False,
)

thigh_adults, _ = process_motus(
    'thigh_adults',
    vendor='Sens',
    map=MAP_THIGH,
    trunk_sensor=False,
    rotate=True,
    orientation=False,
)

_, thigh_trunk_children = process_motus(
    'thigh+trunk_children',
    vendor='Other',
    map=MAP_THIGH_TRUNK,
    trunk_sensor=True,
    rotate=False,
    orientation=False,
)

In [ ]:
labels = ['lie', 'sit', 'stand', 'shuffle', 'walk', 'stairs', 'run', 'bicycle']

In [ ]:
import pandas as pd
from functions import get_confusion_matrix, report

thigh_trunk_children = pd.read_parquet('thigh+trunk_children/processed_thigh+trunk.parquet')
thigh_trunk_children.dropna(subset=['ground_truth'], inplace=True)
thigh_trunk_children['dataset'] = 'thigh_trunk_children'
thigh_trunk_children_cm = get_confusion_matrix(
    true=thigh_trunk_children['ground_truth'],
    pred=thigh_trunk_children['activity'],
    labels=labels,
    title='Thigh/Back (Children, Free-living)',
    color='greens',
    hide_yaxis=False,
    hide_xaxis_title=False,
)

thigh_trunk_children_performance = report(
    thigh_trunk_children, true='ground_truth', pred='activity', labels=labels, subject_id='id'
)
thigh_trunk_children_performance['dataset'] = 'thigh_trunk_children'

thigh_trunk_children_cm

In [ ]:
import pandas as pd
from functions import get_confusion_matrix, report

thigh_adults = pd.read_parquet('thigh_adults/processed_thigh.parquet')
thigh_adults = thigh_adults.loc[thigh_adults['ground_truth'] != 'remove']
thigh_adults['dataset'] = 'thigh_adults'

laboratory = thigh_adults.loc[thigh_adults['condition'] == 'laboratory'].copy()
free_living = thigh_adults.loc[thigh_adults['condition'] == 'free-living'].copy()

laboratory_cm = get_confusion_matrix(
    true=laboratory['ground_truth'],
    pred=laboratory['activity'],
    labels=labels,
    title='Thigh (Adults, Laboratory)',
    color='blues',
    hide_xaxis_title=False,
)


free_living_cm = get_confusion_matrix(
    true=free_living['ground_truth'],
    pred=free_living['activity'],
    labels=labels,
    title='Thigh (Adults, Free-living)',
    color='greens',
    hide_yaxis=True,
    hide_xaxis_title=False,
)

# laboratory_cm.save('thigh_adults_laboratory.png', scale_factor=4)
# free_living_cm.save('thigh_adults_free_living.png', scale_factor=4)

# laboratory_performance = get_validity_metrics(true=laboratory['ground_truth'], pred=laboratory['activity'])
laboratory_performance = report(laboratory, true='ground_truth', pred='activity', labels=labels, subject_id='id')
laboratory_performance['dataset'] = 'thigh_adults_laboratory'

# free_living_performance = get_validity_metrics(true=free_living['ground_truth'], pred=free_living['activity'])
free_living_performance = report(free_living, true='ground_truth', pred='activity', labels=labels, subject_id='id')
free_living_performance['dataset'] = 'thigh_adults_free_living'

In [ ]:
import pandas as pd
from functions import get_confusion_matrix, report

thigh_trunk_adults = pd.read_parquet('thigh+trunk_adults/processed_thigh+trunk.parquet')
thigh_trunk_adults['dataset'] = 'thigh_trunk_adults'
thigh_trunk_adults_cm = get_confusion_matrix(
    true=thigh_trunk_adults['ground_truth'],
    pred=thigh_trunk_adults['activity'],
    labels=labels,
    title='Thigh/Back (Adults, Free-living)',
    color='greens',
    hide_yaxis=True,
    hide_xaxis_title=False,
)

thigh_trunk_older_adults = pd.read_parquet('thigh+trunk_older_adults/processed_thigh+trunk.parquet')
thigh_trunk_older_adults['dataset'] = 'thigh_trunk_older_adults'
thigh_trunk_older_adults_cm = get_confusion_matrix(
    true=thigh_trunk_older_adults['ground_truth'],
    pred=thigh_trunk_older_adults['activity'],
    labels=labels,
    title='Thigh/Back (Older Adults, Free-living)',
    color='greens',
    hide_yaxis=True,
    hide_xaxis_title=False,
)

# thigh_trunk_adults_cm.save('thigh_trunk_adults.png', scale_factor=4)
# thigh_trunk_older_adults_cm.save('thigh_trunk_older_adults.png', scale_factor=4)

# thigh_trunk_adults_performance = get_validity_metrics(
#     true=thigh_trunk_adults['ground_truth'], pred=thigh_trunk_adults['activity']
# )
thigh_trunk_adults_performance = report(
    thigh_trunk_adults, true='ground_truth', pred='activity', labels=labels, subject_id='id'
)
thigh_trunk_adults_performance['dataset'] = 'thigh_trunk_adults'

# thigh_trunk_older_adults_performance = get_validity_metrics(
#     true=thigh_trunk_older_adults['ground_truth'], pred=thigh_trunk_older_adults['activity']
# )
thigh_trunk_older_adults_performance = report(
    thigh_trunk_older_adults, true='ground_truth', pred='activity', labels=labels, subject_id='id'
)
thigh_trunk_older_adults_performance['dataset'] = 'thigh_trunk_older_adults'

In [ ]:
x = (laboratory_cm | free_living_cm | thigh_trunk_adults_cm | thigh_trunk_older_adults_cm).resolve_scale(
    color='independent'
)
x.save('performance.png', scale_factor=4)

performance = pd.concat(
    [
        laboratory_performance,
        free_living_performance,
        thigh_trunk_adults_performance,
        thigh_trunk_older_adults_performance,
    ]
)[labels + ['dataset']]
performance.to_excel('performance.xlsx', index=True)

In [ ]:
labels = ['sedentary', 'stand', 'walk', 'run', 'bicycle']

In [ ]:
import pandas as pd
from functions import get_confusion_matrix

fused = {
    'lie': 'sedentary',
    'sit': 'sedentary',
    'shuffle': 'stand',
    'stairs': 'walk',
}

thigh_adults = pd.read_parquet('thigh_adults/processed_thigh.parquet')
thigh_adults['ground_truth'] = thigh_adults['ground_truth'].replace(fused)
thigh_adults['activity'] = thigh_adults['activity'].astype('string').replace(fused)

laboratory = thigh_adults.loc[thigh_adults['condition'] == 'laboratory'].copy()
free_living = thigh_adults.loc[
    (thigh_adults['condition'] == 'free-living') & (thigh_adults['ground_truth'] != 'remove')
].copy()

laboratory_cm = get_confusion_matrix(
    true=laboratory['ground_truth'],
    pred=laboratory['activity'],
    labels=labels,
    title='Thigh (Adults, Laboratory)',
    color='blues',
    hide_xaxis_title=False,
)


free_living_cm = get_confusion_matrix(
    true=free_living['ground_truth'],
    pred=free_living['activity'],
    labels=labels,
    title='Thigh (Adults, Free-living)',
    color='greens',
    hide_yaxis=True,
    hide_xaxis_title=False,
)

# laboratory_cm.save('thigh_adults_laboratory.png', scale_factor=4)
# free_living_cm.save('thigh_adults_free_living.png', scale_factor=4)

# laboratory_performance = get_validity_metrics(true=laboratory['ground_truth'], pred=laboratory['activity'])
laboratory_performance = report(laboratory, true='ground_truth', pred='activity', labels=labels, subject_id='id')
laboratory_performance['dataset'] = 'thigh_adults_laboratory'

# free_living_performance = get_validity_metrics(true=free_living['ground_truth'], pred=free_living['activity'])
free_living_performance = report(free_living, true='ground_truth', pred='activity', labels=labels, subject_id='id')
free_living_performance['dataset'] = 'thigh_adults_free_living'

In [ ]:
import pandas as pd
from functions import get_confusion_matrix

thigh_trunk_adults = pd.read_parquet('thigh+trunk_adults/processed_thigh+trunk.parquet')
thigh_trunk_adults['ground_truth'] = thigh_trunk_adults['ground_truth'].replace(fused)
thigh_trunk_adults['activity'] = thigh_trunk_adults['activity'].astype('string').replace(fused)

thigh_trunk_adults_cm = get_confusion_matrix(
    true=thigh_trunk_adults['ground_truth'],
    pred=thigh_trunk_adults['activity'],
    labels=labels,
    title='Thigh/Back (Adults, Free-living)',
    color='greens',
    hide_yaxis=True,
    hide_xaxis_title=False,
)

thigh_trunk_older_adults = pd.read_parquet('thigh+trunk_older_adults/processed_thigh+trunk.parquet')
thigh_trunk_older_adults['ground_truth'] = thigh_trunk_older_adults['ground_truth'].replace(fused)
thigh_trunk_older_adults['activity'] = thigh_trunk_older_adults['activity'].astype('string').replace(fused)

thigh_trunk_older_adults_cm = get_confusion_matrix(
    true=thigh_trunk_older_adults['ground_truth'],
    pred=thigh_trunk_older_adults['activity'],
    labels=labels,
    title='Thigh/Back (Older Adults, Free-living)',
    color='greens',
    hide_yaxis=True,
    hide_xaxis_title=False,
)

# thigh_trunk_adults_cm.save('thigh_trunk_adults.png', scale_factor=4)
# thigh_trunk_older_adults_cm.save('thigh_trunk_older_adults.png', scale_factor=4)

# thigh_trunk_adults_performance = get_validity_metrics(
#     true=thigh_trunk_adults['ground_truth'], pred=thigh_trunk_adults['activity']
# )
thigh_trunk_adults_performance = report(
    thigh_trunk_adults, true='ground_truth', pred='activity', labels=labels, subject_id='id'
)
thigh_trunk_adults_performance['dataset'] = 'thigh_trunk_adults'

# thigh_trunk_older_adults_performance = get_validity_metrics(
#     true=thigh_trunk_older_adults['ground_truth'], pred=thigh_trunk_older_adults['activity']
# )
thigh_trunk_older_adults_performance = report(
    thigh_trunk_older_adults, true='ground_truth', pred='activity', labels=labels, subject_id='id'
)
thigh_trunk_older_adults_performance['dataset'] = 'thigh_trunk_older_adults'

In [ ]:
x = (laboratory_cm | free_living_cm | thigh_trunk_adults_cm | thigh_trunk_older_adults_cm).resolve_scale(
    color='independent'
)
x.save('performance_fused.png', scale_factor=4)

performance = pd.concat(
    [
        laboratory_performance,
        free_living_performance,
        thigh_trunk_adults_performance,
        thigh_trunk_older_adults_performance,
    ]
)[labels + ['dataset']]
performance.to_excel('performance_fused.xlsx', index=True)

In [ ]:
thigh_trunk_children = pd.read_parquet('thigh+trunk_children/processed_thigh+trunk.parquet')
thigh_trunk_children.dropna(subset=['ground_truth'], inplace=True)

thigh_trunk_children['ground_truth'] = thigh_trunk_children['ground_truth'].replace(fused)
thigh_trunk_children['activity'] = thigh_trunk_children['activity'].astype('string').replace(fused)

thigh_trunk_children_cm = get_confusion_matrix(
    true=thigh_trunk_children['ground_truth'],
    pred=thigh_trunk_children['activity'],
    labels=labels,
    title='Thigh/Back (Children, Free-living)',
    color='greens',
    hide_yaxis=True,
    hide_xaxis_title=False,
)

thigh_trunk_children_cm